In [ ]:
from ultralytics import YOLO
from sort import *
import cv2
import cvzone
import math

In [ ]:
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

In [ ]:
model = YOLO('yolov8l.pt')
cap = cv2.VideoCapture('Videos/cars.mp4')
mask = cv2.imread("Images/cars_mask.png")   # Load an image that acts as a mask, limiting object detection to a specific area (e.g., in which area we need to count).

In [ ]:
#Tracking

tracker = Sort(max_age=20, min_hits=3 , iou_threshold=0.3)   # # Initialize the SORT (Simple Online and Realtime Tracking) algorithm for tracking objects from github.

# max_age => Number of frames the tracker will keep an object in memory even if it’s not detected (set to 20 frames here).
# min_hits => Minimum number of consecutive frames the object must be detected before tracking starts (set to 3 frames here).
# iou_threshold => Threshold for Intersection over Union (IoU) used to determine when a detection is considered a match to a tracked object (set to 0.3).

In [26]:
# Counting the objects that cross a specific area (defined by the limits).

limits = [400, 297, 673, 297]  # Define the coordinates of a line or region (e.g., x1, y1, x2, y2) that will be used to count objects crossing it.
totalCount = []  # List to store the count of objects that have crossed the specified line or region.

In [ ]:
while True:
    success, img = cap.read()
    imageRegion = cv2.bitwise_and(img,mask)   # Apply the mask to the image using a bitwise AND operation, keeping only the parts of the image that match the mask.
    
    result = model(imageRegion, stream=True)
    detections = np.empty((0,5))    # Initialize an empty NumPy array to store detection results. Each detection will have 5 values: [x_min, y_min, x_max, y_max, confidence_score].
    for r in result:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w = x2 - x1
            h = y2 - y1

            conf =  (math.ceil(box.conf[0] * 100))/100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            if currentClass == "car" and conf > 0.3:  # Check if the detected object is a "car" and the confidence is greater than 0.3.
                currentArray = np.array([x1, y1, x2, y2, conf])  # Create an array with the bounding box coordinates (x1, y1, x2, y2) and the confidence score.
                detections = np.vstack((detections, currentArray))  # Add the new detection to the detections array by stacking it vertically.

    resultsTracker = tracker.update(detections)  # Update the tracker with the current frame's detections. This returns the updated tracking results (e.g., object IDs and their positions).
    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 225), 5)  # Draw a line on the image to define the counting region (using coordinates from 'limits'). The line is red (BGR: 0, 0, 225) and has a thickness of 5 pixels.

    
    for results in resultsTracker:
        x1, y1, x2, y2, Id = results  # Unpack the tracking results: x1, y1, x2, y2 are the bounding box coordinates, and Id is the unique identifier for the tracked object.
        x1, y1, x2, y2, Id = int(x1), int(y1), int(x2), int(y2), int(Id)
        
        print(results)
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255,0,255))
        cvzone.putTextRect(img, f'{Id}' ,(max(0,x1), max(35,y1)), scale = 2, thickness = 3, offset=10)
        
        cx, cy = int(x1 + w / 2), int(y1 + h / 2)  # Calculate the center coordinates (cx, cy) of the bounding box using its top-left corner (x1, y1) and its width (w) and height (h).
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)  # Draw a filled circle at the center (cx, cy) with radius 5, using a magenta color (BGR: 255, 0, 255) on the image.


        if limits[0] < cx < limits[2] and limits[1] - 30 < cy < limits[3] + 30:  # Check if the object's center (cx, cy) is within the defined region (with a buffer of 30 pixels vertically).
            if Id not in totalCount:  # If the object ID has not been counted yet, add it to the totalCount list to avoid double-counting.
                totalCount.append(Id)  # Add the object's ID to the list, indicating it has crossed the counting region.
                cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)  # Draw a green line (BGR: 0, 255, 0) to visually highlight the counting boundary, with thickness 5 pixels.

    cvzone.putTextRect(img, f'Count: {len(totalCount)}' ,(50,50), scale = 2, thickness = 3, offset=10)

    
    cv2.imshow("Video",img)
    cv2.waitKey(1)